# Computation on NumPy Arrays

The key for fast computation on NumPy arrays is the usage of vectorized operations, normally implemented through NumPy's _universal functions_ (ufuncs). NumPy's ufuncs can be used to make repeated calculations on array elements much more efficient.

The following snippet (taken from the handbook) is an example of how slow iterative loops can be on Python, and later is presented a possible solution to that using NumPy arrays

In [1]:
import numpy as np
# Set the seed to some known value for reproducibility
np.random.seed(0)

# Computes the multiplicative inverse for each element and returns an array with the result
def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output

The following piece of code took roughly 2 seconds to execute on my machine. This is absurd slow for today's computes for a simple $O(N)$ operation on a array of one million cells (not small, but not that large either). This is an example of where the runtime type-checking causes massive overhead.

In [2]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

1.99 s ± 31.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


To overcome this issue, NumPy provides statically typed vectorized operations (that is, operations that are applied to each element of the array). The following piece of code is equivalent to the _compute_reciprocals_ function, but much faster. On my machine, the execution time was improved by a factor of roughly 1000 times.

In [3]:
%timeit (1.0 / big_array)

2.12 ms ± 71.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Vectorization through the usage of ufuncs is almost always more efficient than their iterative loops counterparts, especially for large arrays. 

Array arithmetic using ufuncs supports all Python's native arithmetic operators and they are, in the form they were presented previously, simply convenient wrappers to operators implemented in NumPy (e.g. `np.divide`).

Since there are too many to list here, and [this](https://jakevdp.github.io/PythonDataScienceHandbook/02.03-computation-on-arrays-ufuncs.html#Exploring-NumPy's-UFuncs) section of the handbook already does a great job of pointing out useful information about ufuncs, the following notes will be brief and highlight only a few aspects of this section.

## Advanced Ufunc Features

It is possible to directly specify the output array where the result of the calculation will be stored, instead of creating a temporary array for that purpose. This can be done using the `out` argument of the function:

In [4]:
# Sequential array ranging from 0 to 4 inclusive
x = np.arange(5)
# No need to initialize memory here since it will be overwritten anyway
y = np.empty(5)
np.multiply(x, 10, out=y)

array([ 0., 10., 20., 30., 40.])

This also works with arrays views. The following example stores the result of the operating to every other element of `y`:

In [5]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


The usage of the argument `out` here implies that the result is directly stored in the output array, instead of creating a temporary one to store the result of the calculations, to later be copied into the appropriate positions of `y`. This can save significant amounts of memory for very large arrays.

### Aggregates

Some interesting aggregates can be used in addition to an ufunc. For example:

`reduce`: Repeatedly applies a given operation to the elements of an array until only a single element remains

`accumulate`: Stores the intermediate results of the computation

`outer`: Computes the output of all pairs of two different inputs

Note: The examples below are just to illustrate the behavior of the aggregates, although there are dedicated NumPy functions to compute the exact same results.

In [6]:
x = np.arange(1, 6)
# Equivalent to np.sum(x)
np.add.reduce(x)

15

In [7]:
# Equivalent to np.cumsum(x)
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15])

In [8]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

### Data Statistics

NumPy provides a variety of methods to compute statics about the data. NumPy arrays support Python's built-in functions (e.g. `sum` and `max`), but their NumPy counterparts (`np.sum` and `np.max`) are much more efficient. The following is a simple example of some useful methods and a comparison between `np.sum` and `sum` performance.

In [9]:
# Compute some insightful information about data
print("Sum of elements:     ", big_array.sum())
print("Product of elements: ", big_array.prod())
print("Mean of elements:    ", big_array.mean())
print("Standard deviation:  ", big_array.std())
print("Variance:            ", big_array.var())
print("Maximum value:       ", big_array.max())
print("Minimum value:       ", big_array.min())
print("Index of max value:  ", big_array.argmax())
print("Index of min value:  ", big_array.argmin())

Sum of elements:      49988718
Product of elements:  0
Mean of elements:     49.988718
Standard deviation:   28.582147517576004
Variance:             816.9391567164762
Maximum value:        99
Minimum value:        1
Index of max value:   112
Index of min value:   53


In [10]:
%timeit sum(big_array)
# Equivalent to big_array.sum()
%timeit np.sum(big_array)

161 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
792 µs ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
